In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging

# Setup paths
PROJECT_ROOT = os.path.dirname(os.path.dirname(os.getcwd()))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Setup directories
MODEL_DIR = os.path.join(os.getcwd(), 'model')
RESULTS_DIR = os.path.join(os.getcwd(), 'results')
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Imports
from preprocessing.data_container import DataContainer
from utils.evaluation import cindex_score
from models.modelling_process import ModellingProcess
from sksurv.ensemble import RandomSurvivalForest

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [ ]:
# Data configuration
DATA_CONFIG = {
    'use_pca': False,
    'pca_threshold': 0.95,
    'gene_type': 'common_genes',
    'use_imputed': True,
    'use_cohorts': False
}

# Model configuration
MODEL_CONFIG = {
    'params_cv': {
        'model__n_estimators': [100],
        'model__min_samples_split': [6], 
        'model__max_features': ['sqrt'],
        'model__bootstrap' : [False], 
        'model__n_jobs': [-1], 
        'model__random_state': [1234], 
        'model__low_memory' : [True] 
    },
    'refit': True, 
    'do_nested_resampling': False, 
    'path' : RESULTS_DIR, 
    'fname_cv' : 'results_intersect'}

rsf_pipeline_steps = [('model', RandomSurvivalForest())]


In [12]:
mp = ModellingProcess()

In [13]:
mp.prepare_data(DATA_CONFIG, PROJECT_ROOT)

2024-11-29 10:14:08,043 - INFO - Loading data...
2024-11-29 10:16:44,317 - INFO - Loaded data: 1091 samples, 15495 features


In [14]:
nstd_res_result = mp.do_modelling(rsf_pipeline_steps, MODEL_CONFIG)

2024-11-29 10:17:11,316 - INFO - No additional monitoring detected
2024-11-29 10:17:11,319 - INFO - Start model training...
2024-11-29 10:17:11,319 - INFO - Input data shape: X=(1091, 15495)
2024-11-29 10:17:11,322 - INFO - Do HP Tuning for complete model; refit + set complete model
2024-11-29 10:17:11,322 - INFO - Do HP Tuning for complete model


Fitting 9 folds for each of 1 candidates, totalling 9 fits


c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
# Save nested resampling result and retrained model (TODO: Add this to modelling process)

In [15]:
mp.resampling_cmplt.cv_results_

{'mean_fit_time': array([2296.95820708]),
 'std_fit_time': array([596.37276507]),
 'mean_score_time': array([3.08911371]),
 'std_score_time': array([1.99421501]),
 'param_model__bootstrap': masked_array(data=[False],
              mask=[False],
        fill_value=True),
 'param_model__low_memory': masked_array(data=[True],
              mask=[False],
        fill_value=True),
 'param_model__max_features': masked_array(data=['sqrt'],
              mask=[False],
        fill_value=np.str_('?'),
             dtype=object),
 'param_model__min_samples_split': masked_array(data=[6],
              mask=[False],
        fill_value=999999),
 'param_model__n_estimators': masked_array(data=[100],
              mask=[False],
        fill_value=999999),
 'param_model__n_jobs': masked_array(data=[-1],
              mask=[False],
        fill_value=999999),
 'param_model__random_state': masked_array(data=[1234],
              mask=[False],
        fill_value=999999),
 'params': [{'model__bootstrap': 